365 days
163 projects
2 pods per project
26 worker nodes
3 AZs

Answer the questions
--------------------

1. When deploying a project, how often does all of it's pods end up on the same AZ?
2. When destroying an AZ, how many services lose:
    *  Both pods
    *  One pod
    *  No pods

In [1]:
from random import randrange
# Create pods model

def create_pods_for_projects(projects_count, pods_per_project):
    pods = []
    count = 0
    for project_id in range(0, projects_count):
        for pod_id in range(0, pods_per_project):
            pod = {"id": count, "name": pod_id, "project": project_id}
            pods.append(pod)
            count += 1
    # print(f"Pod count: {len(pods)}")
    return pods


def distribute_pods_to_nodes(pods, nodes_count):
    # Assign each pod a node - round robin to simulate anti-affinity
    for i in range(0, len(pods)):
        pod = pods[i]
        pod["node"] = i % nodes_count
    return pods
    

def distribute_nodes_to_azs(nodes_count, azs_count):
    # Now assign each node an AZ
    nodes = dict()

    for i in range(0, nodes_count):

        az_for_node = randrange(0, azs_count)
        # print (az_for_node)
        
        # az_for_node = i % azs_count # round robin
        nodes[i] = {"az": az_for_node}
    return nodes

# print(distribute_nodes_to_azs(24, 3))


# When deploying a project, how often does all of it's pods end up on the same AZ?

In [17]:

def run_experiment(projects_count, pods_per_project, nodes_count, azs_count):
    pods = create_pods_for_projects(projects_count=projects_count, pods_per_project=pods_per_project)
    pods = distribute_pods_to_nodes(pods=pods, nodes_count=nodes_count)
    nodes = distribute_nodes_to_azs(nodes_count=nodes_count, azs_count=azs_count)
    return pods, nodes

def describe_distribution(pods, nodes, azs_count, pods_per_project):
    result = dict()

    for pod in pods:
        # which az is this pod's node in?
        az = nodes[pod['node']]['az']
        pod["az"] = az
        # print(f"Pod {pod['id']} is in project {pod['project']}, node {pod['node']} and AZ {pod['az']}")
        
    # projects with two pods in the same AZ
    for az in range(0, azs_count):
        # find all pods in that AZ
        pods_in_az = list()
        projects_in_az = dict()
        for pod in pods:
            if pod['az'] == az:
                pods_in_az.append(pod)
                c = projects_in_az.get(pod['project'], 0)
                c += 1
                projects_in_az[pod['project']] = c

        # print(f"AZ {az} has {len(pods_in_az)} pods and {len(projects_in_az)} projects in it.")
        dangerous_projects = list()
        for k, v in projects_in_az.items():
            percent = (v / pods_per_project) * 100
            if percent == 100:
                # print(f"Project {k} has {percent}% of it's pods in this AZ")
                dangerous_projects.append(k)
        # print(f"{len(dangerous_projects)} projects with all their pods in this AZ.")
        result[az] = dangerous_projects
    
    # print(projects_in_az)
    # print(pods_in_az)
    return result


experiments = list()
pods_per_project = 2
azs_count = 3
projects_count = 163
for experiment_number in range(0, 10000):
    pods, nodes = run_experiment(projects_count=projects_count, pods_per_project=pods_per_project, nodes_count=26, azs_count=azs_count)
    result = describe_distribution(pods, nodes, azs_count=azs_count, pods_per_project=pods_per_project)
    total_at_risk = 0
    for az, v in result.items():
        total_at_risk += len(v)
    mean_at_risk = total_at_risk/azs_count
    result["mean"] = mean_at_risk
    experiments.append(result)

total = 0
for ex_no in range(0, len(experiments)):
    ex = experiments[ex_no]
    # print(f"{ex_no}. \t{len(ex[0])}\t{len(ex[1])}\t{len(ex[2])}\t -- {ex['mean']:.1f}")
    total = total + ex['mean']

mean = total/len(experiments)
print(f"Over {len(experiments)} runs, on average {mean:0.1f} of {projects_count} projects ended up with all pods in the same AZ. That's {mean/projects_count*100:.1f}% or 1 in {100/(mean/projects_count*100):.0f}.")



Over 10000 runs, on average 18.0 of 163 projects ended up with all pods in the same AZ. That's 11.1% or 1 in 9.
